In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://www.marriott.com/hotel-search.mi"
driver.get(url)
time.sleep(1)


In [3]:
findreserve_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div/section/div[1]/div/div/a')[0]
findreserve_button.click()
time.sleep(1)

In [4]:
brandfilter_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/span[4]')[0]
brandfilter_button.click()
time.sleep(1)

In [5]:
springhill_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[1]/form/div[2]/div[1]/div/fieldset/div[3]/ul/li[3]/a/span')[0]
springhill_button.click()
time.sleep(1)

In [6]:
apply_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[2]/div/div/div/form/button[1]')[0]
apply_button.click()
time.sleep(1)

In [7]:
counter = 1
listurls = []
while counter < 30:
    currenturl = driver.current_url
    print(f'Currently on page: {currenturl}.')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
#     nextbutton1
    hotelcards = soup.find('div', {'class' : 'js-property-list-container map-view-scroll-space'})
#     listurls = []
    try:
        for x in hotelcards.find_all('div', {'class' : 'l-row'}):
            try:
                href = x.find('a').attrs['href']

                if 'QuickView' in href:
                    url = {}
                    url['url'] = 'https://www.marriott.com/' + href
                    url['name'] = x.find('a').attrs['title']
                    url['code'] = x.find('a').attrs['data-marsha']
                    listurls.append(url)
        #             print(url)
        #             print()
        #             print('='*50)
        #             print()
            except:
                pass
    except:
        break
    nextbutton = soup.find_all('div', {'role' : 'navigation'})
    nextbutton1 = 'https://www.marriott.com/'
    for x in nextbutton:
        try:
            if counter == 1:
                button = x.find_all('a')[5].attrs['href']
                nextbutton1 = nextbutton1 + button 
            elif counter == 2 or counter == 12:
                button = x.find_all('a')[6].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter == 3 or counter == 11:
                button = x.find_all('a')[7].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter >= 4 and counter <= 10:
                button = x.find_all('a')[8].attrs['href']
                nextbutton1 = nextbutton1 + button
#             else:
#                 button = x.find_all('a')[5].attrs['href']
#                 nextbutton1 = nextbutton1 + button                
        except:
            continue
    driver.get(nextbutton1)
    counter += 1
listurls_df = pd.DataFrame(listurls)

Currently on page: https://www.marriott.com/search/filterSearch.mi.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=2.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=3.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=4.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=5.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=6.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=7.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=8.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=9.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=10.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=11.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=12.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=13.
Currently on pag

In [8]:
listurls_df

,url,name,code
0,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Dallas Addison/Quorum Drive,DALSH
1,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Albany-Colonie,ALBCO
2,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Albuquerque North/Journal Ce...,ABQNJ
3,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Albuquerque University Area,ABQSA
4,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Alexandria,WASAX
...,...,...,...
497,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Detroit Wixom,DTWSW
498,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Potomac Mills Woodbridge,WASSW
499,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Waco Woodway,WCOSH
500,https://www.marriott.com//search/hotelQuickVie...,SpringHill Suites Wrentham Plainville,BOSSW


In [9]:
hotels = []
for index, row in listurls_df.iterrows():
    name = row['name'].replace(' ','-').replace('/','-').lower()
    code = row['code'].lower()
    url = 'https://www.marriott.com/hotels/travel/' + code + '-' + name
    hotel ={}
    hotel['name'] = name
    hotel['url'] = url
    hotels.append(hotel)
hotels_df = pd.DataFrame(hotels)

In [10]:
hotels_df

,name,url
0,springhill-suites-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalsh-s...
1,springhill-suites-albany-colonie,https://www.marriott.com/hotels/travel/albco-s...
2,springhill-suites-albuquerque-north-journal-ce...,https://www.marriott.com/hotels/travel/abqnj-s...
3,springhill-suites-albuquerque-university-area,https://www.marriott.com/hotels/travel/abqsa-s...
4,springhill-suites-alexandria,https://www.marriott.com/hotels/travel/wasax-s...
...,...,...
497,springhill-suites-detroit-wixom,https://www.marriott.com/hotels/travel/dtwsw-s...
498,springhill-suites-potomac-mills-woodbridge,https://www.marriott.com/hotels/travel/wassw-s...
499,springhill-suites-waco-woodway,https://www.marriott.com/hotels/travel/wcosh-s...
500,springhill-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bossw-s...


In [11]:
# nextbutton = soup.find_all('div', {'role' : 'navigation'})
# nextbutton1 = 'https://www.marriott.com/'
# for x in nextbutton:
#     button = x.find_all('a', {'class' : ''}
#     nextbutton1 = nextbutton1 + button
# nextbutton1

In [12]:
# driver.current_url

In [13]:
# driver.get(nextbutton1)

In [14]:
# driver.current_url

In [15]:
# nextbutton = soup.find_all('div', {'role' : 'navigation'})
# nextbutton1 = 'https://www.marriott.com/'
# for x in nextbutton:
#     button = x.find_all('a')[5].attrs['href']
#     nextbutton1 = nextbutton1 + button
# nextbutton1

In [16]:
# hotelcards = soup.find('div', {'class' : 'js-property-list-container map-view-scroll-space'})
# listurls = []
# for x in hotelcards.find_all('div', {'class' : 'l-row'}):
#     try:
#         href = x.find('a').attrs['href']
        
#         if 'QuickView' in href:
#             url = {}
#             url['url'] = 'https://www.marriott.com/' + href
#             url['name'] = x.find('a').attrs['title']
#             url['code'] = x.find('a').attrs['data-marsha']
#             listurls.append(url)
# #             print(url)
# #             print()
# #             print('='*50)
# #             print()
#     except:
#         pass
    

    
# listurls_df = pd.DataFrame(listurls)
# listurls_df

In [17]:
# hotels = []
# for index, row in listurls_df.iterrows():
#     name = row['name'].replace(' ','-').replace('/','-').lower()
#     code = row['code'].lower()
#     url = 'https://www.marriott.com/hotels/travel/' + code + '-' + name
#     hotel['name'] = name
#     hotel['url'] = url
#     hotels.append(url)
#     print(url)

In [18]:
%%time

hotel_list = []
counter = 0
for index, row in hotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Springhill'
        except:
            hotel['brand'] = 'Springhill'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df       

{'brand': 'Springhill', 'name': 'springhill-suites-dallas-addison-quorum-drive', 'url': 'https://www.marriott.com/hotels/travel/dalsh-springhill-suites-dallas-addison-quorum-drive', 'street': '15255 Quorum Dr.', 'locality': 'Addison', 'state': 'Texas', 'postalcode': '75001', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/dalsh-springhill-suites-dallas-addison-quorum-drive/?directPageRequest=true', 'coordinate2': '32.956978', 'coordinate3': '-96.826257', 'coordinate4': 'N/A', 'description': "[ Settle into stylishly updated accommodations at SpringHill Suites Dallas Addison/Quorum Drive. Our all-suite hotel is a short walk from lush green spaces and the beautiful Addison Circle Park. Explore local attractions within a 3-mile radius, including Galleria Dallas, the Addison Conference and Theatre Centre and top corporations. Following an adventurous outing, unwind in our revitalized suites. You'll feel at home with our free Wi-Fi and separate living areas. Kick-off your morning

{'brand': 'Springhill', 'name': 'springhill-suites-alexandria-old-town-southwest', 'url': 'https://www.marriott.com/hotels/travel/wasva-springhill-suites-alexandria-old-town-southwest', 'street': '2950 Eisenhower Avenue', 'locality': 'Alexandria', 'state': 'Virginia', 'postalcode': '22314', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/wasva-springhill-suites-alexandria-old-town-southwest/?directPageRequest=true', 'coordinate2': '38.802499', 'coordinate3': '-77.080372', 'coordinate4': 'N/A', 'description': "[ Embark on a historic stay at SpringHill Suites Alexandria Old Town/Southwest. Located along I-495, our hotel conveniently places you among dynamic destinations in the nation's capital including the National Science Foundation, George Washington Masonic National Memorial, King Street and Cameron Run Regional Park. Catch up on some rest and relaxation in one of our spacious suites. Kick back on our plush bedding while browsing premium channels on our flat-panel TVs. If

{'brand': 'Springhill', 'name': 'springhill-suites-anaheim-maingate', 'url': 'https://www.marriott.com/hotels/travel/snaao-springhill-suites-anaheim-maingate', 'street': '1160 West Ball Road', 'locality': 'Anaheim', 'state': 'California', 'postalcode': '92802', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/snaao-springhill-suites-anaheim-maingate/?directPageRequest=true', 'coordinate2': '33.817213', 'coordinate3': '-117.928255', 'coordinate4': 'N/A', 'description': "[ Offering lots of little extras, SpringHill Suites Anaheim Maingate is the perfect affordable family-friendly all-suite hotel conveniently located near everything in the DISNEYLAND® Resort Area. With larger, stylish suites that include a mini-fridge and microwave, there's enough space to relax for up to six people after a busy day exploring the Platinum Triangle, Garden Grove or Downtown DISNEY® District. In town for an event at the nearby Anaheim Convention Center? Headed to Angels Stadium of Anaheim to catc

{'brand': 'Springhill', 'name': 'springhill-suites-annapolis', 'url': 'https://www.marriott.com/hotels/travel/bwisu-springhill-suites-annapolis', 'street': '189 Admiral Cochrane Drive', 'locality': 'Annapolis', 'state': 'Maryland', 'postalcode': '21401', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bwisu-springhill-suites-annapolis/?directPageRequest=true', 'coordinate3': '38.972368', 'coordinate4': '-76.549309', 'description': '[ Discover style and space at SpringHill Suites Annapolis.  Designed in partnership with west elm, for your convenience our modern guest suites feature extra room, plush bedding, mini-fridge, sink, microwave and dedicated workspaces .  Guests staying at our all-suite Annapolis hotel will have easy access to Annapolis’ top attractions, just minutes from the US Naval Academy, Navy-Marine Corps Stadium, historic downtown Annapolis, Westfield Mall and more.  Stay connected with free WiFi and benefit from free on-site parking

{'brand': 'Springhill', 'name': 'springhill-suites-ashburn-dulles-north', 'url': 'https://www.marriott.com/hotels/travel/iadas-springhill-suites-ashburn-dulles-north', 'street': '20065 Lakeview Center Plaza', 'locality': 'Ashburn', 'state': 'Virginia', 'postalcode': '20147', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/iadas-springhill-suites-ashburn-dulles-north/?directPageRequest=true', 'coordinate3': '39.059695', 'coordinate4': '-77.450711', 'description': '[ Newly Renovated SpringHill Suites Ashburn Dulles North hotel in Ashburn, VA is located along Route 7, north of Dulles International Airport near the Leesburg Outlets and local wedding venues, &amp; walking distance to Top Golf, iFly, &amp; One Loudoun. Guests traveling on business to GEICO, George Washington University (GWU), Verizon, National Transportation Safety Board (NTSB), Telos, Toll Brothers or VISA will appreciate our complimentary hotel-wide WiFi, pre-scheduled Dulles airport s

...

{'brand': 'Springhill', 'name': 'springhill-suites-west-palm-beach-i-95', 'url': 'https://www.marriott.com/hotels/travel/pbiwi-springhill-suites-west-palm-beach-i-95', 'street': '2437 Metrocentre Boulevard', 'locality': 'West Palm Beach', 'state': 'Florida', 'postalcode': '33407', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/pbiwi-springhill-suites-west-palm-beach-i-95/?directPageRequest=true', 'coordinate3': '26.757223', 'coordinate4': '-80.093974', 'description': '[ Refresh at SpringHill Suites West Palm Beach I-95. Relax in our suites with free Wi-Fi, plush Marriott bedding, and large workspaces. Start your day with selections from our free breakfast buffet, serving coffee, Belgian waffles, healthy yogurts, and more ways to add a little extra energy to your stay. Explore delicious local dining in West Palm Beach with restaurants like Cracker Barrel, Pollo Tropical, and the Cheesecake Factory near our hotel. Be your best self during your stay 

{'brand': 'Springhill', 'name': 'springhill-suites-wheeling-triadelphia-area', 'url': 'https://www.marriott.com/hotels/travel/pitwh-springhill-suites-wheeling-triadelphia-area', 'street': '908 National Road', 'locality': 'Wheeling', 'state': 'West Virginia', 'postalcode': '26003', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/pitwh-springhill-suites-wheeling-triadelphia-area/?directPageRequest=true', 'coordinate3': '40.077422', 'coordinate4': '-80.692679', 'description': '[ SpringHill Suites Wheeling is conveniently located just minutes from Downtown.  This Wheeling, WV hotel offers all that the business and leisure traveler is seeking.  Wheeling is a town of hidden treasures in art, history, museums, theaters, and sports.  Located minutes away from the WesBanco Arena, Capitol Theatre, Wheeling Island Casino, The Highlands Sports Complex, and the Ohio Valley Mall, we provide a central location for your entertainment needs.  All of our studio suit

{'brand': 'Springhill', 'name': 'springhill-suites-wilmington-mayfaire', 'url': 'https://www.marriott.com/hotels/travel/ilmsh-springhill-suites-wilmington-mayfaire', 'street': '1014 Ashes Drive', 'locality': 'Wilmington', 'state': 'North Carolina', 'postalcode': '28405', 'coordinate1': '-77.825838', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Get more out of your next visit to Wilmington, North Carolina, by staying at SpringHill Suites Wilmington Mayfaire. With a prime location across from the shops and restaurants of Mayfaire Town Center. A short drive from downtown Wilmington and Wrightsville Beach, we place you near some of the region's top attractions. Step inside our hotel and let us greet you with friendly service and contemporary amenities. Fuel up for a busy day exploring Wilmington, NC, with a delicious free breakfast or stay active using our hotel's on-site fitness center and outdoor pool. Groups traveling to North Carolina appreciate o

{'brand': 'Springhill', 'name': 'springhill-suites-wisconsin-dells', 'url': 'https://www.marriott.com/hotels/travel/msnsh-springhill-suites-wisconsin-dells', 'street': '461 Wisconsin Dells Parkway', 'locality': 'Wisconsin Dells', 'state': 'Wisconsin', 'postalcode': '53965', 'coordinate1': '43.59546', 'coordinate2': '-89.793891', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Designed with your travel needs in mind, SpringHill Suites Wisconsin Dells is perfect for adventurous getaways and productive business retreats. Experiencing new thrills around our hotel is easy with our premier location near Noah's Ark Waterpark, Mt. Olympus water and theme parks and Kalahari Resort Dells. If rides aren't your thing, delight in a live performance at Crystal Grand Music Theatre and seek natural beauty at nearby Lake Delton. Unwind in our spacious accommodations with modern designs, plush bedding, sleek workstations and Smart TVs offering access to your favorite Netflix and Hulu serie

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Springhill,springhill-suites-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalsh-s...,15255 Quorum Dr.,Addison,Texas,75001,https://www.marriott.com/hotels/maps/travel/da...,32.956978,-96.826257,N/A,[ Settle into stylishly updated accommodations...,N/A,4.5,4.5
1,Springhill,springhill-suites-albany-colonie,https://www.marriott.com/hotels/travel/albco-s...,8 California Avenue,Albany,New York,12205,https://www.marriott.com/hotels/maps/travel/al...,42.702435,-73.815405,N/A,[ Catch a breath of fresh air at our upscale h...,N/A,4.5,4.5
2,Springhill,springhill-suites-albuquerque-north-journal-ce...,https://www.marriott.com/hotels/travel/abqnj-s...,5910 Holly Avenue NE,Albuquerque,New Mexico,87113,-106.579844,N/A,N/A,N/A,[ Seize the day at SpringHill Suites Albuquerq...,N/A,4.8,4.7
3,Springhill,springhill-suites-albuquerque-university-area,https://www.marriott.com/hotels/travel/abqsa-s...,1101 Central Avenue NE,Albuquerque,New Mexico,87106,N/A,N/A,N/A,N/A,[ SpringHill Suites by Marriott Albuquerque Un...,N/A,N/A,N/A
4,Springhill,springhill-suites-alexandria,https://www.marriott.com/hotels/travel/wasax-s...,6065 Richmond Highway,Alexandria,Virginia,22303,Reviews,https://www.marriott.com/hotels/maps/travel/wa...,38.785063,-77.070213,[ Who says you can't enjoy great style for an ...,N/A,4.5,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Springhill,springhill-suites-detroit-wixom,https://www.marriott.com/hotels/travel/dtwsw-s...,49097 Alpha Drive,Wixom,Michigan,48393,Reviews,https://www.marriott.com/hotels/maps/travel/dt...,42.500617,-83.534867,[ Experience the modern style of SpringHill Su...,N/A,4.8,4.8
498,Springhill,springhill-suites-potomac-mills-woodbridge,https://www.marriott.com/hotels/travel/wassw-s...,14325 Crossing Place,Woodbridge,Virginia,22192,Reviews,https://www.marriott.com/hotels/maps/travel/wa...,38.654885,-77.283934,"[ Located right off I-95 south of Washington, ...",N/A,4.3,4.3
499,Springhill,springhill-suites-waco-woodway,https://www.marriott.com/hotels/travel/wcosh-s...,200 Colonnade Parkway,Woodway,Texas,76712,Reviews,https://www.marriott.com/hotels/maps/travel/wc...,31.503953,-97.200719,[ Discover the perfect blend of comfort and co...,N/A,4.8,4.7
500,Springhill,springhill-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bossw-s...,65 Ledgeview Way,Wrentham,Massachusetts,02093,N/A,N/A,N/A,N/A,[ SpringHill Suites Wrentham is located only a...,N/A,N/A,N/A


In [19]:
hotel_df = hotel_df.drop_duplicates(subset='name')
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Springhill,springhill-suites-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalsh-s...,15255 Quorum Dr.,Addison,Texas,75001,https://www.marriott.com/hotels/maps/travel/da...,32.956978,-96.826257,N/A,[ Settle into stylishly updated accommodations...,N/A,4.5,4.5
1,Springhill,springhill-suites-albany-colonie,https://www.marriott.com/hotels/travel/albco-s...,8 California Avenue,Albany,New York,12205,https://www.marriott.com/hotels/maps/travel/al...,42.702435,-73.815405,N/A,[ Catch a breath of fresh air at our upscale h...,N/A,4.5,4.5
2,Springhill,springhill-suites-albuquerque-north-journal-ce...,https://www.marriott.com/hotels/travel/abqnj-s...,5910 Holly Avenue NE,Albuquerque,New Mexico,87113,-106.579844,N/A,N/A,N/A,[ Seize the day at SpringHill Suites Albuquerq...,N/A,4.8,4.7
3,Springhill,springhill-suites-albuquerque-university-area,https://www.marriott.com/hotels/travel/abqsa-s...,1101 Central Avenue NE,Albuquerque,New Mexico,87106,N/A,N/A,N/A,N/A,[ SpringHill Suites by Marriott Albuquerque Un...,N/A,N/A,N/A
4,Springhill,springhill-suites-alexandria,https://www.marriott.com/hotels/travel/wasax-s...,6065 Richmond Highway,Alexandria,Virginia,22303,Reviews,https://www.marriott.com/hotels/maps/travel/wa...,38.785063,-77.070213,[ Who says you can't enjoy great style for an ...,N/A,4.5,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Springhill,springhill-suites-detroit-wixom,https://www.marriott.com/hotels/travel/dtwsw-s...,49097 Alpha Drive,Wixom,Michigan,48393,Reviews,https://www.marriott.com/hotels/maps/travel/dt...,42.500617,-83.534867,[ Experience the modern style of SpringHill Su...,N/A,4.8,4.8
498,Springhill,springhill-suites-potomac-mills-woodbridge,https://www.marriott.com/hotels/travel/wassw-s...,14325 Crossing Place,Woodbridge,Virginia,22192,Reviews,https://www.marriott.com/hotels/maps/travel/wa...,38.654885,-77.283934,"[ Located right off I-95 south of Washington, ...",N/A,4.3,4.3
499,Springhill,springhill-suites-waco-woodway,https://www.marriott.com/hotels/travel/wcosh-s...,200 Colonnade Parkway,Woodway,Texas,76712,Reviews,https://www.marriott.com/hotels/maps/travel/wc...,31.503953,-97.200719,[ Discover the perfect blend of comfort and co...,N/A,4.8,4.7
500,Springhill,springhill-suites-wrentham-plainville,https://www.marriott.com/hotels/travel/bossw-s...,65 Ledgeview Way,Wrentham,Massachusetts,02093,N/A,N/A,N/A,N/A,[ SpringHill Suites Wrentham is located only a...,N/A,N/A,N/A


In [20]:
hotel_df.to_csv('springhill_description.csv')